#### Change the promting according to the promting of pip-sql-1.3b
https://huggingface.co/PipableAI/pip-sql-1.3b

#### Go from using langchain sql chain to manual promt template.

In [1]:
from sqlalchemy import create_engine, text
from langchain.sql_database import SQLDatabase

db_path = '/home/arseniy/Документы/Projects/Python/LLM_project/json-to-sql/mys_db'

engine = create_engine ('sqlite:///' + db_path, echo= False, future=True)

db = SQLDatabase(engine)


In [2]:
from langfuse.callback import CallbackHandler
langfuse_handler = CallbackHandler(
    public_key="key",
    secret_key="key",
    host="https://cloud.langfuse.com"
)

In [3]:
from langchain_community.chat_models import ChatLlamaCpp 
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
import multiprocessing

# Path to the model weights
models_directory = '/home/arseniy/Документы/Projects/Python/LLM_project/models/'

local_model = models_directory + 'pip-sql-1.3b.Q8_0.gguf' 


llm = ChatLlamaCpp(
    model_path = local_model,
    temperature = 0.01,
    n_ctx=10000,
    repeat_penalty = 2,
    n_threads=8,
    n_gpu_layers=22,
    use_mlock = True,
    versbose = True,

)


llama_model_loader: loaded meta data with 25 key-value pairs and 219 tensors from /home/arseniy/Документы/Projects/Python/LLM_project/models/pip-sql-1.3b.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = pip-sql-1.3b
llama_model_loader: - kv   2:                          llama.block_count u32              = 24
llama_model_loader: - kv   3:                       llama.context_length u32              = 16384
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5504
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 16
llama_model_loader: - kv  

In [4]:
from operator import itemgetter
from langchain.chains import create_sql_query_chain
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

memory = ConversationBufferMemory()
#sql_chain = create_sql_query_chain(llm, db)


In [5]:
fewShot_promting = True
check_query = True
enable_vector_db = True

In [6]:
examples =[
    {
        "input": "List all projects.",
        "query": """
            SELECT * FROM mysProjects;
        """
    },
    {
        "input": "Find all projects managed by 'Jolene Duran'.",
        "query": """
            SELECT * FROM mysProjects WHERE manager = 'Jolene Duran';
        """
    },
    {
        "input": "List all projects with project code 'MY0849'.",
        "query": """
            SELECT * FROM mysProjects WHERE project_code = 'MY0849';
        """
    },
    {
        "input": "Find all employees in the 'finance & accounting' hierarchy level.",
        "query": """
            SELECT * FROM orgHierarchy WHERE hierarchy_level = 'finance & accounting';
        """
    },
    {
        "input": "List all work histories for 'İstanbul Bilgi Üniversitesi Tercih Günler'.",
        "query": """
            SELECT * FROM workHistory WHERE company = 'İstanbul Bilgi Üniversitesi Tercih Günler';
        """
    },
    {
        "input": "Find all certifications with the name 'Ms Project'.",
        "query": """
            SELECT * FROM certification WHERE name = 'Ms Project';
        """
    },
    {
        "input": "List all comments given by 'Knight Eaton'.",
        "query": """
            SELECT * FROM comments WHERE evaluator = 'Knight Eaton';
        """
    },
    {
        "input": "Find all employees who attended 'Erciyes Üniversitesi'.",
        "query": """
            SELECT * FROM educations WHERE school = 'Erciyes Üniversitesi';
        """
    },
    {
        "input": "List all employees.",
        "query": """
            SELECT * FROM employees;
        """
    },
    {
        "input": "Find all employees who are not absent.",
        "query": """
            SELECT * FROM employees WHERE is_absent = 0;
        """
    },
    {
        "input": "List all languages spoken by employee with id '1'.",
        "query": """
            SELECT * FROM languages WHERE employee_id = 1;
        """
    },
    {
        "input": "Find all leave records for employee with id '2'.",
        "query": """
            SELECT * FROM leaves WHERE employee_id = 2;
        """
    },
    {
        "input": "List all score details for score id '1'.",
        "query": """
            SELECT * FROM score_details WHERE score_id = 1;
        """
    },
    {
        "input": "Find all scores evaluated by 'Eve Mccarty'.",
        "query": """
            SELECT * FROM scores WHERE evaluator = 'Eve Mccarty';
        """
    },
    {
        "input": "List all software skills for employee with id '1'.",
        "query": """
            SELECT * FROM softwares WHERE employee_id = 1;
        """
    },    
    {
        "input": "List all employees along with their total number of certifications.",
        "query": """
            SELECT e.*, COUNT(c.id) as certification_count 
            FROM employees e
            LEFT JOIN certification c ON e.id = c.employee_id
            GROUP BY e.id;
        """
    },
    {
        "input": "Find all employees who have received comments with a score higher than '4' in the last year.",
        "query": """
            SELECT e.* 
            FROM employees e
            JOIN comments cm ON e.id = cm.employee_id
            WHERE cm.score > 4 AND cm.date_time >= DATE('now', '-1 year');
        """
    },
    {
        "input": "List all employees who have obtained a certification in 'Project Management' and are currently not absent.",
        "query": """
            SELECT e.* 
            FROM employees e
            JOIN certification c ON e.id = c.employee_id
            WHERE c.name = 'Project Management' AND e.is_absent = 0;
        """
    },
    {
        "input": "Find the total number of employees in each directorate.",
        "query": """
            SELECT directorate, COUNT(*) as total_employees 
            FROM employees
            GROUP BY directorate;
        """
    },
    {
        "input": "List all employees who have completed their education in the 'Computer Science' department with a 'Bachelor' degree.",
        "query": """
            SELECT e.* 
            FROM employees e
            JOIN educations ed ON e.id = ed.employee_id
            WHERE ed.department = 'Computer Science' AND ed.degree = 'Bachelor';
        """
    },
    {
        "input": "Find all employees who have worked in more than one company.",
        "query": """
            SELECT e.*, COUNT(w.id) as companies_count
            FROM employees e
            JOIN workHistory w ON e.id = w.employee_id
            GROUP BY e.id
            HAVING companies_count > 1;
        """
    },
    {
        "input": "List all employees along with their average comment score.",
        "query": """
            SELECT e.*, AVG(cm.score) as average_score
            FROM employees e
            LEFT JOIN comments cm ON e.id = cm.employee_id
            GROUP BY e.id;
        """
    },
    {
        "input": "Find all employees who have both certifications in 'Data Science' and 'Machine Learning'.",
        "query": """
            SELECT e.*
            FROM employees e
            JOIN certification c1 ON e.id = c1.employee_id AND c1.name = 'Data Science'
            JOIN certification c2 ON e.id = c2.employee_id AND c2.name = 'Machine Learning';
        """
    },
    {
        "input": "List all employees who are older than 40 years and have a certification that expired last year.",
        "query": """
            SELECT e.*
            FROM employees e
            JOIN certification c ON e.id = c.employee_id
            WHERE e.age > 40 AND c.end_date BETWEEN DATE('now', '-1 year') AND DATE('now');
        """
    },
    {
        "input": "Find all employees who have completed education in 'Engineering' and have received at least three certifications.",
        "query": """
            SELECT e.*
            FROM employees e
            JOIN educations ed ON e.id = ed.employee_id AND ed.department = 'Engineering'
            JOIN certification c ON e.id = c.employee_id
            GROUP BY e.id
            HAVING COUNT(c.id) >= 3;
        """
    }
]



In [7]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

dialect = 'SQLite'

#example_prompt =  PromptTemplate.from_template("User input: {input}\nSQL query: {query}")

if (fewShot_promting):
    prompt = FewShotPromptTemplate(
        examples=examples,
        #example_prompt=example_prompt,
        prefix = "You are a SQLite expert. Given an input question, create a syntactically correct SQLite query to run. Unless otherwise specified, do not return more than {top_k} rows. Here is the relevant database schema info: {table_info}. If filtering on a feature value make sure to check its spelling. Always check the spelling of column names! Write only SQL query! Dont write anything else! Try to keep SQL query short. Write SQL query only about the question user asked.",
        
                        
        suffix="User input: {input}\nSQL query: ",
        input_variables=["input", "top_k", "table_info"],
    )
    query_chain = create_sql_query_chain(llm, db, prompt)
    
    if(enable_vector_db):
        retriever_chain = (
        itemgetter("question")
        | retriever
        | (lambda docs: "\n".join(doc.page_content for doc in docs))
        )
        chain = RunnablePassthrough.assign(emails=retriever_chain) | query_chain
else:
    query_chain = create_sql_query_chain(llm, db)

ValidationError: 1 validation error for FewShotPromptTemplate
example_prompt
  field required (type=value_error.missing)

In [ ]:
with open('description.txt', 'r') as file:
    description = file.read()


In [ ]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

dialect = 'SQLite'

example_prompt =  PromptTemplate.from_template("User input: {input}\nSQL query: {query}")

if (fewShot_promting):
    prompt = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
        prefix = "You are a SQLite expert. Given an input question, create a syntactically correct SQLite query to run. Unless otherwise specified, do not return more than {top_k} rows. Here is the relevant database schema info: {table_info}. If filtering on a feature value make sure to check its spelling. Check the provided description for the database: {table_info} .Always check the spelling of column names! Write only SQL query! Dont write anything else! Try to keep SQL query short. Write SQL query only about the question user asked.",           
        suffix="User input: {input}\nSQL query: ",
        input_variables=["input", "top_k", "table_info", "description"],
    )
    query_chain = create_sql_query_chain(llm, db, prompt)
    
    if(enable_vector_db):
        retriever_chain = (
        itemgetter("question")
        | retriever
        | (lambda docs: "\n".join(doc.page_content for doc in docs))
        )
        chain = RunnablePassthrough.assign(emails=retriever_chain) | query_chain
else:
    query_chain = create_sql_query_chain(llm, db)

In [ ]:
def convert_query_result_to_text(prompt, query_result):
    # Convert query result to text
    result_text = f"The query result is: {query_result}"
    
    # Create input for the chain that will interpret the query result
    interpretation_prompt = f"Based on the user question: '{prompt}' and database query result: '{query_result}', interpret the results. Write it very simple words, without any technical details. Write very short. Dont exceed one sentence. Make it understandable for human without knowing tecnical details."
    
    # Use another chain to interpret the query result
    interpretation = llm2.invoke(interpretation_prompt)
    
    return interpretation


In [ ]:
import sqlvalidator
import sqlcorrect as sc

def main():
    print("🔴 Simple database chatbot app. Type 'exit' to quit.")
    while True:
        # Get user input
        question = input("🟢 Enter your question: ")
        #memory.chat_memory.add_user_message(question)
        if question.lower() == 'exit':
            print("Exiting the app.")
            break
        
        # Invoke the chain with the user's question
        # if check_query = True, then query checking operation will be performed.
        if (check_query): 
            response = full_chain.invoke({"question": question}, input_variables={"vocabulary": email},config={"callbacks": [langfuse_handler]} ) 
        elif (enable_vector_db): 
            response = chain.invoke({"question": question + " Check the spelling of attributes in the question from the lsit of values which you have! Especially check the spelling of names, emails, titles!!! Write only the SQL query, nothing more! "}, config={"callbacks": [langfuse_handler]}) 
        else:
            response = query_chain.invoke({"question": question }, config={"callbacks": [langfuse_handler]}) 

        #corrected_query = sc.correct_sql_query(response)
        print("Response:", response)
        
        sql_query = sqlvalidator.format_sql(response)
        print("Formated SQL:" + sql_query)
        
        #memory.chat_memory.add_ai_message(sql_query) 
        try:
            db_answer = db.run(sql_query)
            print(db_answer)
            #rint(convert_query_result_to_text(question, db_answer))
        except:
            print("🔴 LLM agent gave a wrong SQL query. Paraphrase the question...")
            #print(convert_query_result_to_text(question, db_answer))
        

if __name__ == "__main__":
    main()